# Econometria Aplicada

## Regressão linear múltipla

João Ricardo Costa Filho \\
[joaocostafilho.com](https://)

## A regressão linear múltipla

**Motivação**: \\

O que explica o comércio bilateral entre os países?

O exercício quantitativo desta aula é baseado em [Gravity in R: a short workshop](https://imedkrisna.github.io/gravity/).

### Dados

Vamos utilizar os seguintes conjuntos de dados:

* Gravity: "Data to estimate gravity equations: trade, GDP, population, trade agreements, etc. For all country pairs, 1948-2020."

Eles estão disponíveis neste [link](https://www.cepii.fr/CEPII/en/bdd_modele/bdd_modele.asp). Iremos baixar a versão para o R:

In [ ]:
url      <- "https://www.cepii.fr/DATA_DOWNLOAD/gravity/legacy/202202/Gravity_rds_V202202.zip"

In [ ]:
destfile <- "Gravity_rds_V202202.zip"

In [ ]:
download.file( url, destfile, mode = "wb")

In [ ]:
unzip( destfile )

Podemos verificar no diretório no qual estamos trabalhando que os dados foram baixados. Para isso, clique na 'pasta' no menu vertical que está à esquerda.

Façamos o upload da base de dados:

In [ ]:
gravity <- readRDS("Gravity_V202202.rds")

Faremos alguns filtros. O primeiro, diz respeito ao ano dos dados. Trabalhemos com 2019, antes da pandemia. Depois, filtraremos os códigos dos páises de origem e de destino, respectivamente. Finalmente, excluíremos dos dados referentes à situação na qual o país de origem e destino é o mesmo:

In [ ]:
library(dplyr)

In [ ]:
gravity2 <- gravity %>% filter( year == 2019 ) %>%
            filter( country_id_o!="IDN.1" )    %>%
            filter( country_id_d!="IDN.1" )    %>%
            filter( iso3_o!=iso3_d )

Obtenha as primeiras 10 linhas da base de dados 'gravity2':

Obtenha a estrutura dos dados da base 'gravity2':

Criemos dentro do dataframe 'gravity2' algumas variáveis:

In [ ]:
gravity2 <- gravity2 %>% mutate( lndist    = log( distw_harmonic ),      # Distância (média harmônica)
                                 lngdpo    = log( gdp_o ),               # PIB do país de origem
                                 lngdpd    = log( gdp_d ),               # PIB do país de destino
                                 lngdpco   = log( gdpcap_o ),            # PIB per capita do país de origem
                                 lngdpcd   = log( gdpcap_d ),            # PIB per capita do país de destino
                                 lntrade = log( 1 + tradeflow_baci )     # Fluxo de comércio entre dois países
                                 )

## Visualização dos dados

Faça uma dispersão entre o log fluxo de comércio entre dois países e o log PIB do país de origem:

Estatísticas descritivas

### Regressão

Estime a seguinte regressão: \\
\
$\ln T_{i,j} = \beta_0 + \beta_1 \ln PIB_{o, i} + ɛ_i$


Qual é o resultado da estimativa? (Ou seja, como é a equação com os valores estimados?)

A relação entre os fluxos comerciais bilaterais e o PIB do país de origem é estatísticamente significativa? Qual é o parâmetro estimado que devemos testar para responder essa pergunta?

E a significância conjunta?

Adicione a linha da regressão na disperão anterior:

Sendo assim, qual é o valor do fluxo comercial biltareal **esperado** de um país de origem cujo PIB é igual à 1.000.000.000?

### Resíduos

Vamos olhar para as estatísticas referentes aos resíduos agora.

In [ ]:
trade_hat = fitted( reg )

e_hat = resid( reg )

Por hipótese, temos que $E[\varepsilon] = 0$

In [ ]:
round( mean( e_hat ), 10 )

Por hipótese, temos que $Cov[ \varepsilon_i, X_i ] = 0 \implies \rho( \varepsilon_i,  X_i) = 0$

In [ ]:
# Dados utilizados na regressão:
model_data <- model.frame(reg)

lngdpo_used <- model_data$lngdpo

round( cor( lngdpo_used, e_hat), 10)

Faça um histograma com os resíduos da regressão:

### Regressão linear múltipla

Faça uma dispersão entre o log fluxo de comércio entre dois países e o log PIB do país de destino:

Como podemos fazer para estimar a equação abaixo? \\

$T_{o,d} = \frac{ Y^{\beta_1}_o \times Y^{\beta_2}_d }{ D^{\beta_3}_{o,d} }$ \\

onde $T_{i,j}$ representa o fluxo comercial bilateral entre o país de origem ($o$) e o país de destino ($d$), $Y_o$ é o PIB do país de origem, $Y_d$ é o PIB do país de destino e $D_{o,d}$ a distância entre eles.



Queremos estimar a seguinte regressão: \\
\
$\ln T_{i,j} = \beta_1 Y_o + \beta_2 Y_d + \beta_3 \ln Dist_{i,j} + ɛ_i$


O que eu espero dos coeficientes da regressão?

$\beta_1 > 0$, $\beta_2 > 0$ e $\beta_3 < 0$.

Estime a regressão (no que não há intercepto nela!):

#### Resíduos

Vamos olhar para as estatísticas referentes aos resíduos agora.

Por hipótese, temos que $E[\varepsilon] = 0$

Por hipótese, temos que $Cov[ \varepsilon_i, X_i ] = 0 \implies \rho( \varepsilon_i,  X_i) = 0$

Faça um histograma com os resíduos da regressão:

* Dummy

In [ ]:
reg <- lm( lntrade ~ 0 + lngdpo + lngdpd + lndist + wto_o + wto_d, data=gravity2 )

summary( reg )

# Extra: Teorema Frisch-Waugh-Lovell

In [ ]:
# 1) Faça a regressão de lntrade em lngdpd, lndist, wto_o, wto_d e obtenha os resíduos:



In [ ]:
# 2) Faça a regressão de lngdpo em lngdpd, lndist, wto_o, wto_d e obtenha os resíduos:



In [ ]:
# 3) Faça a regressão dos resíduos da regressão do passo 1 nos resíduos da regressão do passo 2:



Note que o coeficiente dos resíduos é o mesmo coeficiente da variável 'lngdpo' na regressão anterior.

## Extre 2: exportar os resultados para Latex (não faz parte do escopo do curso)

In [ ]:
install.packages("stargazer")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(stargazer)

In [ ]:
reg <- lm( lntrade ~ 0 + lngdpo + lngdpd + lndist, data=gravity2 )

In [ ]:
stargazer(reg,
  type = "latex",
  title = "Resultados da Regressão",
  dep.var.labels = "Log do Comércio (lntrade)",
  covariate.labels = c("Ln PIB (Origem)", "Ln PIB (Destino)", "Ln Distância"),
  omit.stat = c("f", "ser"),
  digits = 3,
  header = FALSE,
  column.sep.width = "1pt",
  font.size = "scriptsize"
)

In [ ]:
reg <- lm( lntrade ~ 0 + lngdpo + lngdpd + lndist + wto_o + wto_d, data=gravity2 )

In [ ]:
stargazer(reg,
  type = "latex",
  title = "Resultados da Regressão",
  dep.var.labels = "Log do Comércio (lntrade)",
  covariate.labels = c("Ln PIB (Origem)", "Ln PIB (Destino)", "Ln Distância",
                       "Membro da OMC (Origem)", "Membro da OMC (Destino)"),
  omit.stat = c("f", "ser"),
  digits = 3,
  header = FALSE,
  column.sep.width = "1pt",
  font.size = "scriptsize"
)